In [78]:
import mysql.connector
from mysql.connector import errorcode
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import glob
from mysql.connector.constants import ClientFlag
import shutil
from datetime import timedelta
import time

In [57]:
u_name = 'root'
pass_word = 'Vo1cijesuislalumiere?'

In [58]:
cnx = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Vo1cijesuislalumiere?"
)

mycursor = cnx.cursor()
mycursor.execute("DROP DATABASE IF EXISTS OPM;")
mycursor.execute("CREATE DATABASE IF NOT EXISTS OPM;")
cnx.close()

In [59]:
cnx = mysql.connector.connect(user=u_name, password=pass_word, host='localhost', database='OPM')
cursor = cnx.cursor(buffered=True)
cursor.execute("DROP TABLE IF EXISTS opm_fact;")
cursor.execute("CREATE TABLE IF NOT EXISTS opm_fact (AGYSUB VARCHAR(4), LOC VARCHAR(2), AGELVL VARCHAR(1), EDLVL VARCHAR(4), GSEGRD VARCHAR(4), LOSLVL VARCHAR(1),\
                OCC VARCHAR(4), PATCO VARCHAR(1), PP VARCHAR(2), PPGRD VARCHAR(5), SALLVL VARCHAR(1), STEMOCC VARCHAR(4), SUPERVIS VARCHAR(2), TOA VARCHAR(2), WORKSCH VARCHAR(1),\
                WORKSTAT VARCHAR(1), DATECODE DATE, EMPLOYMENT VARCHAR(1), SALARY FLOAT(12,2), LOS FLOAT(12,2));")

In [60]:
# cnx = mysql.connector.connect(user=u_name, password=pass_word, host='localhost', database='OPM',client_flags=[ClientFlag.LOCAL_FILES])
# cursor = cnx.cursor(buffered=True)
# cursor.execute("DROP TABLE IF EXISTS opm_fact;")
# cursor.execute("CREATE TABLE IF NOT EXISTS opm_fact (AGYSUB VARCHAR(4), LOC VARCHAR(2), AGELVL VARCHAR(1), EDLVL VARCHAR(4), GSEGRD VARCHAR(4), LOSLVL VARCHAR(1),\
#                 OCC VARCHAR(4), PATCO VARCHAR(1), PP VARCHAR(2), PPGRD VARCHAR(5), SALLVL VARCHAR(1), STEMOCC VARCHAR(4), SUPERVIS VARCHAR(2), TOA VARCHAR(2), WORKSCH VARCHAR(1),\
#                 WORKSTAT VARCHAR(1), DATECODE DATE, EMPLOYMENT VARCHAR(1), SALARY FLOAT(12,2), LOS FLOAT(12,2));")

In [61]:
data_files = []
for file in glob.glob("./Data/OPM_data/*.txt"):
    data_files.append(file)

In [62]:
# cursor.execute('SHOW VARIABLES WHERE Variable_Name = "datadir";')
# dir_path = cursor.fetchone()[1]

In [63]:
# original = r"./Data/OPM_data"
# target = dir_path + 'OPM_data'

# shutil.move(original, target)

In [64]:
# cursor.execute(f"LOAD DATA INFILE '{data_files[0]}' INTO TABLE opm_fact FIELDS TERMINATED BY ',' ENCLOSED BY '\"' LINES TERMINATED BY '\\n'")

In [65]:
dtype_dict = {'AGYSUB':str, 'LOC':str, 'AGELVL':str, 'EDLVL':str, 'GSEGRD':str, 'LOSLVL':str, 'OCC':str, 'PATCO':str, 'PP':str, 'PPGRD':str, 'SALLVL':str, 'STEMOCC':str, 
             'SUPERVIS':str, 'TOA':str, 'WORKSCH':str, 'WORKSTAT':str, 'DATECODE':str, 'EMPLOYMENT':str, 'SALARY':str, 'LOS':float}

In [66]:
query = """INSERT INTO opm_fact (AGYSUB, LOC, AGELVL, EDLVL, GSEGRD, LOSLVL, OCC, PATCO, PP, PPGRD, SALLVL, STEMOCC, SUPERVIS, TOA, WORKSCH, WORKSTAT,\
           DATECODE, EMPLOYMENT, SALARY, LOS) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s )"""

In [80]:
for j in range(5):
    starttime = time.perf_counter()
    df = pd.read_csv(data_files[j], dtype=dtype_dict)
    df.iloc[:,-2] = df.iloc[:,-2].str.replace('$', '')
    df.iloc[:,-2] = df.iloc[:,-2].str.replace(',', '')
    df.iloc[:,-2] = df.iloc[:,-2].astype('float')
    df.fillna(-1, inplace=True)
    if df.shape[1] == 19:
            df.iloc[:, 15] = pd.to_datetime(df.iloc[:, 15], format='%Y%m')

            for i in range(df.shape[0]):
                cursor.execute(query, (df.iloc[i,0],df.iloc[i,1],df.iloc[i,2],df.iloc[i,3],df.iloc[i,4],df.iloc[i,5],df.iloc[i,6],df.iloc[i,7],'-1',df.iloc[i,8],df.iloc[i,9],df.iloc[i,10]
                                   ,df.iloc[i,11],df.iloc[i,12],df.iloc[i,13],df.iloc[i,14],df.iloc[i,15],df.iloc[i,16],df.iloc[i,17],df.iloc[i,18]))
    else:
        df.iloc[:, 16] = pd.to_datetime(df.iloc[:, 16], format='%Y%m')
        for i in range(df.shape[0]):
            cursor.execute(query, (df.iloc[i,0],df.iloc[i,1],df.iloc[i,2],df.iloc[i,3],df.iloc[i,4],df.iloc[i,5],df.iloc[i,6],df.iloc[i,7],df.iloc[i,8],df.iloc[i,9],df.iloc[i,10],df.iloc[i,11]
                                   ,df.iloc[i,12],df.iloc[i,13],df.iloc[i,14],df.iloc[i,15],df.iloc[i,16],df.iloc[i,17],df.iloc[i,18],df.iloc[i,19]))
    cnx.commit()
    duration = timedelta(seconds=time.perf_counter()-starttime)
    print(j, " - ", data_files[j], ": ", duration)
cnx.close()
    

KeyboardInterrupt: 

In [68]:
df.head()

,AGYSUB,LOC,AGELVL,EDLVL,GSEGRD,LOSLVL,OCC,PATCO,PPGRD,SALLVL,STEMOCC,SUPERVIS,TOA,WORKSCH,WORKSTAT,DATECODE,EMPLOYMENT,SALARY,LOS
0,AB00,FR,I,13,14,H,1630,2,GS-14,I,XXXX,2,20,F,2,200912,1,94573.0,26.2
1,AB00,FR,I,04,13,H,0801,1,GS-13,H,0801,8,10,F,1,200912,1,84739.0,25.0
2,AB00,UK,H,13,12,E,1630,2,GS-12,F,XXXX,2,10,F,1,200912,1,65320.0,11.2
3,AB00,UK,J,06,11,F,1630,2,GS-11,E,XXXX,2,10,F,1,200912,1,54497.0,18.0
4,AB00,FR,K,10,12,F,1630,2,GS-12,F,XXXX,2,10,F,1,200912,1,67299.0,16.2


In [70]:
df1 = pd.read_csv('./Data/OPM_data/FACTDATA_SEP1998.TXT')
df1.head()

C:\Users\kevin\AppData\Local\Temp\ipykernel_2208\3925946926.py:1: DtypeWarning: Columns (3,4,6,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('./Data/OPM_data/FACTDATA_SEP1998.TXT')


,AGYSUB,LOC,AGELVL,EDLVL,GSEGRD,LOSLVL,OCC,PATCO,PPGRD,SALLVL,STEMOCC,SUPERVIS,TOA,WORKSCH,WORKSTAT,DATECODE,EMPLOYMENT,SALARY,LOS
0,AB00,IT,I,04,11.0,F,1630,2,GS-11,D,XXXX,2,20,F,2,199809,1,"$42,709",18.7
1,AB00,RP,G,04,8.0,D,1630,2,GS-08,C,XXXX,2,20,F,2,199809,1,"$31,045",5.8
2,AB00,BE,H,10,9.0,C,1630,2,GS-09,C,XXXX,8,20,F,2,199809,1,"$31,266",4.9
3,AB00,FR,E,17,13.0,E,810,1,GS-13,E,0810,2,20,F,2,199809,1,"$57,393",12.8
4,AB00,NL,J,04,12.0,E,1630,2,GS-12,Z,XXXX,2,10,F,1,199809,1,NaN,11.7
